In [1]:
#ссылка на мой репозиторий в git hub: https://github.com/angelica-t/HSE-Machine-Learning
import pandas as pd
import os
import json
import numpy as np
from pandas.io.json import json_normalize


#some_data=data.sample(frac=0.1, random_state=2)
#print(some_data.dtypes)
data = pd.read_csv("./train.csv", low_memory=False)
df = data.sample(frac=0.1, random_state=2)
df_s = data.sample(frac=0.01, random_state=2)

df.to_csv(r'10%_data.csv', index = False)
df_s.to_csv(r'1%_data.csv', index = False)

In [2]:
def load_df(csv_path='./1%_data.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [3]:
df_train = load_df()

Loaded 1%_data.csv. Shape: (9037, 54)


In [59]:
df_train.to_csv(r'clean1%_data.csv', index = False)
print(df_train.dtypes)

channelGrouping             object
date                         int64
fullVisitorId               object
sessionId                   object
socialEngagementType        object
visitId                      int64
visitNumber                  int64
visitStartTime               int64
device.browser              object
device.operatingSystem      object
device.isMobile               bool
device.deviceCategory       object
geoNetwork.continent        object
geoNetwork.subContinent     object
geoNetwork.country          object
geoNetwork.networkDomain    object
totals.visits               object
totals.hits                 object
totals.pageviews            object
totals.newVisits            object
trafficSource.source        object
trafficSource.medium        object
dtype: object


In [107]:
revenue=getcol['totals.transactionRevenue'].fillna(0.0).astype(float)
df_train['Revenues from transactions']=revenue

In [121]:
from datetime import datetime
df_train['visitStartTime']=pd.to_datetime(df_train['visitStartTime'], unit='s')
df_train['totals.pageviews']=df_train['totals.pageviews'].fillna(0.0).astype(float)
browser=df_train['device.browser'].astype('category')
operSystem=df_train['device.operatingSystem'].astype('category')
subcont=df_train['geoNetwork.subContinent'].astype('category')
channel=df_train['channelGrouping'].astype('category')
engagedOrNot=df_train['socialEngagementType'].astype('category')

In [125]:
# ГРУППИРОВКА И ФУНКЦИИ
def agg(group): # функция, в которой считаются все показатели
    freeday=0
    daytime=0
    revenueForUser=sum(group['Revenues from transactions'])
    pageViewsForUser=sum(group['totals.pageviews'])
    numberOfDevices=group['device.operatingSystem'].nunique()
    for index, row in group.iterrows(): # пробегаемся по датам в каждой группе 
        if row['visitStartTime'].strftime('%A')=='Saturday' or row['visitStartTime'].strftime('%A')=='Sunday':
            freeday=1 #выходной день или нет
        if 5<=row['visitStartTime'].hour<=21:
            daytime=1 #дневное или ночное время
            
    return freeday, daytime, revenueForUser, pageViewsForUser, numberOfDevices
def func(group): #функция, в которой все посчитанные показатели собираются в один массив
    freeday,daytime, revenueForUser,pageViewsForUser,numberOfDevices=agg(group)
    d = {}
    d['Weekend or not'] = freeday
    d['Daytime or not'] = daytime
    d['Revenue']=revenueForUser
    d['Page views']=pageViewsForUser
    d['Number of devices']=numberOfDevices
    return pd.Series(d, index=['Weekend or not','Daytime or not','Revenue','Page views','Number of devices'])
groupedData = df_train.groupby("fullVisitorId").apply(func)
groupedData= groupedData.reset_index()
groupedData['Browser']=browser.cat.codes
groupedData['Operating system']=operSystem.cat.codes
groupedData['Part of the world']=subcont.cat.codes
groupedData['Channel types']=channel.cat.codes
groupedData['Socially engaged']=engagedOrNot.cat.codes
groupedData

,fullVisitorId,Weekend or not,Daytime or not,Revenue,Page views,Number of devices,Browser,Operating system,Part of the world,Channel types,Socially engaged
0,000005103959234087,1.0,0.0,0.0,8.0,1.0,5,9,17,7,0
1,0003546642773393141,1.0,1.0,0.0,1.0,1.0,5,9,6,4,0
2,0005627692133034833,0.0,1.0,0.0,1.0,1.0,5,9,7,7,0
3,0007194843978514987,0.0,1.0,0.0,1.0,1.0,19,6,14,7,0
4,0007514108108860221,0.0,1.0,0.0,1.0,1.0,5,9,11,4,0
...,...,...,...,...,...,...,...,...,...,...,...
8967,9997870904571299374,0.0,0.0,0.0,1.0,1.0,19,12,11,2,0
8968,9998025013142890030,0.0,0.0,0.0,1.0,1.0,5,9,7,2,0
8969,9998030024650532978,0.0,0.0,0.0,1.0,1.0,5,9,11,4,0
8970,9998650597961901505,1.0,0.0,0.0,1.0,1.0,5,3,11,4,0


In [123]:
df_train

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.operatingSystem,...,geoNetwork.subContinent,geoNetwork.country,geoNetwork.networkDomain,totals.visits,totals.hits,totals.pageviews,totals.newVisits,trafficSource.source,trafficSource.medium,Revenues from transactions
0,Social,20161129,324272247457982354,324272247457982354_1480453021,Not Socially Engaged,1480453021,1,2016-11-29 20:57:01,Chrome,Windows,...,Southern Europe,Portugal,vodafone.pt,1,3,3.0,1,youtube.com,referral,0.0
1,Organic Search,20170317,5322552539294131300,5322552539294131300_1489817299,Not Socially Engaged,1489817299,1,2017-03-18 06:08:19,Chrome,Windows,...,Eastern Asia,Hong Kong,netvigator.com,1,1,1.0,1,google,organic,0.0
2,Social,20161003,860949384581209567,860949384581209567_1475520229,Not Socially Engaged,1475520229,1,2016-10-03 18:43:49,Chrome,Windows,...,Eastern Europe,Russia,avangarddsl.ru,1,4,3.0,1,youtube.com,referral,0.0
3,Social,20161113,6907002564702847740,6907002564702847740_1479085291,Not Socially Engaged,1479085291,1,2016-11-14 01:01:31,Safari,Macintosh,...,Southeast Asia,Thailand,3bb.co.th,1,1,1.0,1,youtube.com,referral,0.0
4,Organic Search,20170731,5482421667155283216,5482421667155283216_1501528076,Not Socially Engaged,1501528076,1,2017-07-31 19:07:56,Chrome,Windows,...,Northern America,United States,express-scripts.com,1,3,3.0,1,google,organic,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9032,Organic Search,20161210,8315448591315069391,8315448591315069391_1481392216,Not Socially Engaged,1481392216,1,2016-12-10 17:50:16,Chrome,Windows,...,Western Europe,Netherlands,planet.nl,1,3,3.0,1,google,organic,0.0
9033,Referral,20161030,2272874700536231601,2272874700536231601_1477860613,Not Socially Engaged,1477860613,1,2016-10-30 20:50:13,Chrome,Macintosh,...,Northern America,United States,comcast.net,1,5,4.0,1,mall.googleplex.com,referral,0.0
9034,Referral,20160805,2345032900568838936,2345032900568838936_1470416833,Not Socially Engaged,1470416833,1,2016-08-05 17:07:13,Chrome,Macintosh,...,Northern America,United States,minterfield.com,1,2,2.0,1,siliconvalley.about.com,referral,0.0
9035,Social,20161127,3156576754571344653,3156576754571344653_1480263155,Not Socially Engaged,1480263155,1,2016-11-27 16:12:35,Edge,Windows,...,Northern America,United States,unknown.unknown,1,8,8.0,1,youtube.com,referral,0.0
